# Monte Carlo methods

Don't forget to save your progress during the datalab to avoid any loss due to crashes.

In this datalab we are going to get acquainted with the basics of Monte Carlo particle transport methods, and we will learn how to sample random events and random values from various distributions.

The new python knowledge from the lab is going to be 
- random number generators from `numpy.random`

Let's get started and have some fun!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Statistics
Any given result from a Monte Carlo calculation, colloquially known as a “tally”, represents an estimate of the mean of some random variable of interest. This random variable typically corresponds to some physical quantity like a reaction rate, a net current across some surface, or the neutron flux in a region. Given that all tallies are produced by a stochastic process, there is an associated uncertainty with each value reported. It is important to understand how the uncertainty is calculated and what it tells us about our results. To that end, we will introduce a number of theorems and results from statistics that should shed some light on the interpretation of uncertainties.

### Law of Large Numbers

The law of large numbers is a fundamental statistical theorem that states that the average value of the results of a large number of repeated experiments will converge to the expected value as the number of trials increases.

Let $ X_1, X_2, \dots, X_n$ be an infinite sequence of independent, identically-distributed random variables with expected values $ E(X_1) = E(X_2) = \mu $. 

One form of the law of large numbers states that the sample mean 

$$
\overline{X}_n = \frac{X_1 + \cdots + X_n}{n}
$$

converges in probability to the true mean $ \mu $. That is, for all $ \epsilon > 0 $:

$$
P(|\overline{X}_n - \mu| > \epsilon) \to 0 \quad \text{as } n \to \infty.
$$

This result highlights that as the number of trials $ n $ increases, the sample mean becomes a more accurate estimate of the population mean.

In [ ]:
# Sample size
N = 1000000

# Simulate rolling a die
true_mean_die = 3.5  # The true mean of a fair six-sided die
rolls = np.random.randint(1, 7, size=N)  # Generate N rolls of a six-sided die

# Compute the running mean
running_mean_die = np.cumsum(rolls) / np.arange(1, N + 1)

difference_die = # TODO Compute the absolute difference between the estimated mean and the true mean


fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot the running mean
# TODO Plot the running mean and the true mean
ax[0].set_xlabel('Sample Size')
ax[0].set_ylabel('Mean')
ax[0].set_xscale('log')
ax[0].set_title('Convergence of Sample Mean to True Mean (Die Rolls)')
ax[0].legend()
ax[0].grid()

# Plot the difference
# TODO Plot the absolute difference between running mean and true mean
# TODO plot the 1/sqrt(N) line for comparison
ax[1].set_xlabel('Sample Size')
ax[1].set_ylabel('Absolute Difference')
ax[1].set_xscale('log')
ax[1].set_yscale('log')
ax[1].set_title('Difference Between Estimated and True Mean (Die Rolls)')
ax[1].legend()
ax[1].grid()

fig.tight_layout()
plt.show()


### Central Limit Theorem (CLT)

The Central Limit Theorem (CLT) is one of the most important results in probability and statistics. It describes the behavior of the sample mean of a large number of independent, identically-distributed random variables.

The CLT states that, given a sequence of independent and identically-distributed random variables $ X_1, X_2, \dots, X_n $ with:

- Expected value $ E(X_i) = \mu $
- Variance $ \text{Var}(X_i) = \sigma^2 < \infty $

the distribution of the sample mean $ \overline{X}_n = \frac{1}{n} \sum_{i=1}^n X_i $ will approach a normal distribution as the sample size $ n \to \infty $, regardless of the original distribution of the $ X_i $.

More formally, the random variable:

$$
Z = \sqrt{n} \left( \overline{X}_n - \mu \right) / \sigma
$$

converges in distribution to the standard normal distribution $ N(0, 1) $ as $ n \to \infty $.
#### Key Implications:
1. The CLT allows us to make inferences about population parameters using sample statistics, even if the population distribution is not normal.
2. It is the foundation for many statistical methods, including hypothesis testing and confidence intervals.

#### Example:
If we repeatedly take samples of size $ n $ from a population with mean $ \mu $ and variance $ \sigma^2 $, the distribution of the sample means will approximate a normal distribution as $ n $ becomes large.

In [ ]:
# Number of dice to roll
num_dice = # TODO Define the number of dice to roll in a list

# Number of trials for each case
num_throws = 100000

# Plot the distributions
plt.figure(figsize=(12, 8))

for i, n in enumerate(num_dice):
    # Simulate rolling 'n' dice 'num_throws' times
    sample_means = np.mean(np.random.randint(1, 7, size=(num_throws, n)), axis=1)
    
    # Plot the histogram of the sums
    plt.subplot(2, 3, i + 1)
    # TODO plot a histogram of the sample mean
    plt.title(f'{n} Dice')
    plt.xlabel('Sample Mean')
    plt.ylabel('Probability Density')

plt.tight_layout()
plt.show()

In [ ]:
# If the die is biased, the distribution of the sample mean will also converge to a normal distribution, but the mean and variance will be different.

# Number of samples to draw
num_samples = # TODO Define the number of dice to roll in a list

# Number of trials for each case
numb_throws = 1000

# Plot the distributions
fig, axs = plt.subplots(1, len(num_samples), figsize=(15, 4))

for i, n in enumerate(num_samples):
    # Simulate drawing 'n' samples from a non-normal distribution 'num_trials' times
    sample_means = [np.mean(np.random.choice([1, 2, 3, 4, 5, 6], p=[0.6, 0.2, 0.1, 0.05, 0.025, 0.025], size=n)) for _ in range(numb_throws)]
    
    # Plot the histogram of the sample means
    # TODO plot a histogram of the sample mean
    axs[i].set_title(f'{n} Samples')
    axs[i].set_xlabel('Sample Mean')
    axs[i].set_ylabel('Probability Density')

plt.tight_layout()
plt.show()

## Sampling from distributions

Let's apply this to a more neutronically relevant observable...

### Discrete distribution: which event happens?

The probability of reaction $i$ happening at energy $E$ is 

\begin{equation}
\frac{\Sigma_i(E)}{\Sigma_t(E)}
\end{equation}

Let us consider that in our material only two reactions might happen: scattering or capture, thus a simple condition can be used to decide which happens.

Complete the `reactionType` function to return a random event type. Assume that at the energy the neutron is travelling with $\Sigma_s=0.64 \: \text{cm}^{-1}$ and $\Sigma_c=0.39 \: \text{cm}^{-1}$. Call the function with these values.

In [ ]:
def reactionType(SigS,SigC):
    """Function to sample a random event type
    
    Parameters
    ----------
    SigS : float
        Macroscopic scattering cross section
    SigC : float
        Macroscopic capture cross section
    """
    SigT= # TODO Complete the line
    x= #TODO Sample random number between 0 and 1
    if x < SigS/SigT:
        return 'scatter'
    # TODO else return 'capture'
    #

ss=0.64
sc=0.39
print()# TODO Complete the line with the function call

Numpy actually has a built in function `np.random.choice()`, which does the same for us. As an input it takes a list of choices to sample from, and optionally one can also pass a list of probabilities.

In [ ]:
np.random.choice(['scatter','capture'],p=[ss/(ss+sc),sc/(ss+sc)])

### Continous distribution I: path to next collision

Let's consider that we have some probability density function $p(x)$, and the related cumulative distribution function is $F(x)=\int_{-\infty}^xp(t)dt$. This function is going to take values between 0 and 1. So if we can sample random numbers uniformly between 0 and 1, we could just convert them by evaluating the inverse of the cumulative distribution function to obtain a random value $x$ sampled from the distribution:

$x=F^{-1}(r)$

This of course is only useful, when it is possible to easily integrate the probability density function.  

Let's see how we can use this to sample random distances travelled by a neutron between collision events. We learnt that 

$\exp(-\Sigma_t x)$ is the probability that a neutron moves a distance dx without any interaction.

and 

$\Sigma_t \exp(-\Sigma_t x)dx$ is the probability that the neutron has its interaction at dx.

So

$p(x)=\Sigma_t \exp(-\Sigma_t x)$

Thus

$F(x)=1-\exp(\Sigma_tx)$

If we take the inverse, to sample a random path

$x=-\frac{\ln(1-r)}{\Sigma_t}$

but if r is uniform over $[0,1]$, than $1-r$ is also uniform over $[0,1]$, so this simplifies to

$x=-\frac{\ln r}{\Sigma_t}$

Complete the `distanceToCollision` function below.

**Note #1** computational speed is everything in MC calculations. Although in this course we don't try to avoid every unnecessary operation, this example is just to highlight that sometimes operations can be avoided with some reasoning.

**Note #2** the natural logarithm can be computed with `np.log`.

**Note #3** `numpy.random` has a built-in function to sample the exponential distribution, nevertheless here we will convert the uniformly distributed random numbers between $[0,1]$ to exponentially distributed random numbers.

In [ ]:
def distanceToCollision(SigT,N=1):
    """Function to sample the distance between collisions
    
    Parameters
    ----------
    SigT : float
        Total Macroscopic cross section in 1/cm
    N : int
        Number of events to be sampled (default=1)
    
    Returns
    -------
    x : float or array-like
        Random distance between collisions
    """
    r = np.random.uniform(0,1,N)
    x = # TODO Complete the line
    return x

We can now try this function. Let's consider that 1 MeV neutrons enter a material which has a total cross section of $\Sigma_t=0.18 \: \text{cm}^{-1}$ at this energy. Or well, let's consider that 10k neutrons enter the material, and let's see how the distribution of the random distances looks like, and what is the mean.

In [ ]:
SigT=0.18
N=10000
ds= # TODO Call distanceToCollision() here

plt.figure()
plt.hist(ds,100)
plt.show()

print('Empirical Mean (cm) \t Theoretical mean (cm)')
print() # TODO Print the empirical mean free path. and the mean free path expected from theory

### Continous distribution II: Watt distribution


When the probability density function is less well behaving, and we cannot obtain the cumulative distribution function easily, we can use for example the rejection method. In this case, we draw a random number (r1), convert it to be between $a$ and $b$ (the bounds of the random value), then we draw an other random number (r2) to create a $y$ value based on the maximum of the probaility density function (M). If the $(x,y)$ pair is under the curve (ie. $y<p(x)$) we accept the value. 

<img src="rejection.png" width="200"/>

**Note** This might be very inefficient if the probability density function is peaked. There are several other methods to more efficient sampling.

Let's try to use this method for sampling the Watt-spectrum which is the probability density function of the energy of neutrons emerging from fission.

$$\chi (E)=C_1\cdot \exp(-\frac{E}{C_2})\cdot \sinh(\sqrt{C_3\cdot E})$$

Draw 100 numbers $x$ between 0-10 MeV and draw 100 numbers $y$ between 0 and the maximum of $\chi(E)$. If the sampled energy is accepted, plot the $(x,y)$ coordinate with green, else with red.

Does this method seem to be efficient to sample the Watt-spectrum? Count the number of accepted random samples to estimate the efficiency!

In [ ]:
def watt(x): 
    C1 = 0.453
    C2 = 0.965
    C3 = 2.29
    return # TODO Complete the line
                                
E=np.linspace(0,10,10000)
plt.figure()
plt.plot(E,watt(E))
maxW=np.max(watt(E))

for _ in range(100):
    xi=np.random.uniform(0,10)
    yi= # TODO Complete the line
    if yi<watt(xi):
        plt.plot(xi,yi,'gx')
    # TODO Complete the if/else statements
    # TODO Count how often a number is accepted!
    
plt.xlabel('Energy (MeV)')
plt.ylabel(r'$\chi (MeV^{-1})$')
plt.show()

print() # TODO Print the estimated efficiency 